In [28]:
from pylogic.assumptions_context import AssumptionsContext, conclude
from pylogic.proposition.proposition import Proposition
from pylogic.structures.set_ import SeqSet
from pylogic.theories.numbers import Naturals
from pylogic.proposition.quantified.forall import ForallInSet
from pylogic.variable import Variable
from pylogic.expressions.prod import Prod
from pylogic.expressions.mod import Mod
from pylogic.proposition.not_ import Not, neg
from pylogic.proposition.ordering.greaterthan import GreaterThan
from pylogic.proposition.ordering.greaterorequal import GreaterOrEqual
from pylogic.constant import Constant

zero = Constant(0)
one = Constant(1)

In [29]:
def disp_prop_list(props):
    for a in props:
        display(a)

In [ ]:


with AssumptionsContext() as ctx:
    m = ctx.var('m')
    m.is_in(Naturals).assume()
    u = ctx.var('u', sequence=True, finite=True, natural=True, positive=True, length=m)
    u_set = SeqSet(u)
    u_set_is_subset_naturals = u_set.is_subset_of(Naturals).by_inspection()

    k = Variable('k')
    all_elements_prime = ForallInSet(k, Naturals, Naturals.prime(u[k])).assume()

    product_u = Prod(u)

    N = product_u + 1
    # proving N > 1:
    # since N is (a positive number) + 1, N > 1
    n_gt_1 = Naturals.theorems.order_theorems.add_positive_gt_parts(product_u, one)(GreaterThan(product_u, zero).by_inspection(), GreaterThan(one, zero).by_inspection())[1]

    # all u_k > 1
    all_elements_gt_1 = all_elements_prime.definition.extract_conjuncts()[0]

    # forall t in {u_n}: N mod t = 1
    with AssumptionsContext() as ctx3:
        t = ctx3.var('t')
        t_in_u_set = t.is_in(u_set).assume()
        k, (_, t_eq_uk) = t_in_u_set.thus_predicate()

        # the mod gives 1 mod u_k, so we need to show that u_k > 1 to just get 1
        uk_gt_1 = all_elements_gt_1(k)
        uk = uk_gt_1.left
        n_mod_t_eq_1 = (N % uk).equals(1).by_simplification().substitute("left", t_eq_uk)
        conclude(n_mod_t_eq_1)
    each_p_in_s_remainder_1 = ctx3.get_proven()[0]
    
    # N is some product of primes by the fundamental theorem of arithmetic
    # the sequwnce of primes is called s here
    n_product_of_primes = Naturals.theorems.prime_theorems.gt_1_is_prod_primes(N)(n_gt_1)
    s, (_, _, all_si_prime, n_eq_prod) = n_product_of_primes
    prod_sn = n_eq_prod.right
    # s_0 is prime but s_0 is not in {u_n}
    # since N mod s_0 = 0 => N mod s_0 != 1,
    # but for all t in {u_n}, N mod t = 1
    s0_prime = all_si_prime(0)
    s0_gt_1 = s0_prime.definition.extract()[0]
    s0 = s0_gt_1.left
    s0_neq_0 = Naturals.theorems.prime_theorems.prime_neq_0(s0)(s0_prime)
    prod_sn_mod_s0 = (prod_sn % s0)
    prod_sn_mod_s0_eq_0 = prod_sn_mod_s0.equals(0).by_simplification()
    prod_sn_mod_s0_neq_1 = neg(zero.equals(one)).by_inspection().substitute("left", prod_sn_mod_s0_eq_0)
    n_mod_s0_neq_1 = prod_sn_mod_s0_neq_1.substitute("left", n_eq_prod)
    s0_in_set_u_implies_n_mod_s0_eq_1 = each_p_in_s_remainder_1.in_particular(s0)
    s0_not_in_set_u = n_mod_s0_neq_1.modus_tollens(s0_in_set_u_implies_n_mod_s0_eq_1)

    exists_prime_not_in_u = s0_prime.and_(s0_not_in_set_u).thus_there_exists("q", s0, Naturals)
    conclude(exists_prime_not_in_u)
    
ctx.get_proven()[0]

here


True

ForallInSet(Variable(m, deps=()), Set_Naturals, ForallInSet(Variable(u, deps=()), Set_AllFiniteSequences, Implies(And(ForallInSet(Variable(n, deps=()), Set_Naturals, IsContainedIn(u_(n), Naturals)), ForallInSet(Variable(k, deps=()), Set_Naturals, Prime(u_(k)))), ExistsInSet(Variable(q, deps=()), Set_Naturals, And(Prime(q), Not(IsContainedIn(q, {u})))))))